Модель двигателя: $\ddot{\theta} + \frac{1}{T} \dot{\theta} = \frac{k}{T} U$

При нулевых начальных условиях, и входном воздействии $u$ (Step response) после преобразования Лапласа имеем:

$$
    s^2 \Theta(s) + s \frac{1}{T} \Theta(s) = \frac{k}{T} \frac{U_0}{s}
$$

$$
    \Theta(s) = \frac{k U_0}{s^2 (Ts + 1)}
$$

Разложим на слагаемые:

$$
    \Theta(s) = \frac{kU_0}{s^2} + \frac{kU_0T^2}{Ts + 1} + \frac{kU_0T}{s}
$$

Выполним обратное преобразрование Лапласа:

$$
    \theta(t) = kU_0 \cdot t + kU_0T \cdot e^{-\frac{1}{T} t} - kU_0T
$$

$$
    \theta(t) = kU_0(t - T \cdot (1-e^{-\frac{1}{T} t}))
$$

In [3]:
import numpy as np
from scipy.optimize import least_squares
import matplotlib.pyplot as plt
import os
import shutil

DATA_PATH = "../data/task1"
FILES = os.listdir(DATA_PATH)

PLOT_PATH = "figs"
if os.path.exists(PLOT_PATH):
    shutil.rmtree(PLOT_PATH)
os.makedirs(PLOT_PATH)

In [16]:
def angle_optimization(x, angle, U_volts, time):
    return angle - angle(x, U_volts, time)


def angle(x, U_volts, time):
    return U_volts * x[0] * (time - x[1] * (1 - np.exp(-time / x[1])))


ks = []
Ts = []

In [ ]:
for file in FILES:
    data =  np.loadtxt(f"{DATA_PATH}/{file}", dtype=float)
    time = data[:, 0]
    angle = data[:, 1] * np.pi / 180
    speed = data[:, 2] * np.pi / 180 
    u = data[:, 3] * data[:, 4]
    
    initial_guess = np.array([114.49, 0.077]) 
    
    optimized_guess = least_squares(angle_optimization, initial_guess, args=(angle, u, time))
    ks.append(optimized_guess.x[0])
    Ts.append(optimized_guess.x[1])

k_mean, T_mean = np.mean(ks), np.mean(Ts)
k_mean, T_mean 

In [ ]:
for indx, file in enumerate(FILES):
    data =  np.loadtxt(f"{DATA_PATH}/{file}", dtype=float)
    time = data[:, 0]
    angle = data[:, 1] * np.pi / 180
    speed = data[:, 2] * np.pi / 180 
    u = data[:, 3] * data[:, 4]
    
    angle_optimized = angle((ks[indx], Ts[indx]), u, time)
    angle_optimized_mean = angle((k_mean, T_mean), u, time)
    
    angle_optimized_error = np.sum((angle_optimized - angle)**2)
    angle_optimized_mean_error = np.sum((angle_optimized_mean - angle)**2)
    
    plt.plot(time, angle, 'r', label='Data')
    plt.plot(time, angle_optimized, 'g--', label='Optimization')
    plt.plot(time, angle_optimized_mean, 'b--', label='Optimization Mean')
    
    plt.title(f'U = {u: .2f}V; k, T = {optimized_guess[0]: .3f}, {optimized_guess[1]: .3f}\n E = {angle_optimized_error: .2f}; E_mean = {angle_optimized_mean_error: .2f}')
    plt.xlabel('t, c')
    plt.ylabel('angle(t)')
    plt.legend()
    plt.imsave(f'{PLOT_PATH}/{u}.jpg')
    plt.clf()